In [ ]:
import matplotlib
matplotlib.use("PDF")
from pylab import *
from scipy import stats
thedir='/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole14/'
%cd $thedir
from scipy.special import lambertw, erf
from scipy.optimize import minimize, fmin_powell


In [ ]:
def FOM(params):
    fom = 0.0
    [A, B, C, D] = params
   
    for i,ne in enumerate(Ne):
        qfe = QFe[i]
        ne_model = A * real(lambertw(exp(C * ( (B - qfe) - D * tanh(B - qfe)))))
        #print ne, ne_model
        if ne < 1.0:
            continue
        else:
            fom += (ne - ne_model)**2
    return fom


In [ ]:
clf()
QFe = []
Ne = []
lowQFe = []
lowNe = []
highQFe = []
highNe = []
file = open("qfe35.txt")
lines = file.readlines()
for line in lines:
    items = line.split()
    qfe = float(items[15].strip(','))
    ne = float(items[12].strip(','))
    print "QFe = %f, Ne = %d"%(qfe, ne)
    if qfe > 15.0:
        continue
    QFe.append(qfe)
    Ne.append(ne)
    if ne < 100.0:
        lowQFe.append(qfe)
        lowNe.append(ne)
    if ne > 20000.0:
        highQFe.append(qfe)
        highNe.append(ne)
lowslope, intercept, r_value, p_value, std_err = stats.linregress(lowQFe, log(lowNe))
print lowslope, intercept, r_value
highslope, intercept, r_value, p_value, std_err = stats.linregress(highQFe, highNe)
print highslope, intercept, r_value
A = 1891
B = 9.02
C = 46.30
D = 0.727
param0 = [A, B, C, D]
print FOM(param0)

args = ()    
Result = fmin_powell(FOM, param0, args)
print Result
[A, B, C, D] = Result

clf()
figure()
subplots_adjust(hspace = 1.0, wspace = 1.0)
xplot4 = linspace(2.0,15.0,500)
yplot4 = A * real(lambertw(exp(C * ( (B - xplot4) - D * tanh(B - xplot4)))))
#yplot4 = A * lambertw(exp(C * (B - xplot4)))
yplot5 = -A * C * (1.0 - D / cosh(B-xplot4)**2) * real(lambertw(exp(C * ( (B - xplot4) - D * tanh(B - xplot4))))) / (real(lambertw(exp(C * ( (B - xplot4) - D * tanh(B - xplot4))))) + 1)
for i in range(40):
    qfe = 5.0 + 0.25 * i
    ne =  A * real(lambertw(exp(C * ( (B - qfe) - D * tanh(B - qfe)))))
    print "qfe = %.2f, Ne = %.3f"%(qfe, ne)
subplot(2,2,1)
scatter(QFe, log10(Ne))
plot(xplot4, log10(yplot4), color = 'magenta', ls = '--')
xlabel("Quasi-Fermi Level(V)")
ylabel("Number of Electrons")
ylim(0,6)
subplot(2,2,2)
scatter(QFe, Ne)
plot(xplot4, yplot4, color = 'magenta', ls = '--')
xlabel("Quasi-Fermi Level(V)")
ylabel("Number of Electrons")
subplot(2,2,3)
plot(xplot4, yplot5, color = 'red', ls = '--')
xlabel("Quasi-Fermi Level(V)")
ylabel("Slope")
savefig("QFE_Ne6.pdf")

In [ ]:
clf()
figure()
subplots_adjust(hspace=0.5, wspace=0.5)
for (filename, res) in [("qfe35.txt",2)]:#, ("qfe61.txt",2)]:
    QFe = []
    Ne = []
    file = open(filename)
    lines = file.readlines()
    for line in lines:
        items = line.split()
        qfe = float(items[15].strip(','))
        ne = float(items[12].strip(','))
        if qfe > 15.0:
            continue
        QFe.append(qfe)
        Ne.append(ne)
    fit = polyfit(QFe, Ne,2)
    print fit
    xplot4 = linspace(4.0,12.0,500)

    A = fit[0]; B = fit[1]; C = fit[2]#; D = fit[3]
    yplot4 = A * xplot4 * xplot4 + B * xplot4 + C# * xplot4 + D

    fit2 = polyfit(Ne, QFe,2)
    print fit2
    yplot5 = linspace(0.0,300000.0,500)

    A = fit2[0]; B = fit2[1]; C = fit2[2]#; D = fit2[3]
    xplot5 = A * yplot5 * yplot5 + B * yplot5 + C# * yplot5 + D
    
    slope = 2.0 * A * xplot4 + B
    A = 20000.0; B = -400000.0; C = 2000000.0
    for nne in range(1,100000,5000):
        sl = -sqrt(B*B - 4*A*(C-nne))
        #sl = 3.0E-6*nne**2 - 1.14*nne
        print "Ne = %d, slope = %.1f"%(nne, sl)
    sl = -sqrt(B*B - 4*A*(C-yplot4))
    subplot(1,2,1)
    title("Number of Electrons vs Quasi-Fermi Level. SF = %d"%res, fontsize = 6)
    scatter(QFe,Ne)
    plot(xplot4, yplot4, color = 'blue', ls = '--')
    xlim(4.0,11.0)
    subplot(1,2,2)
    title("Slope", fontsize = 6)

    plot(xplot5, yplot5, color = 'magenta', ls = '--')
    xlim(0, 200000)
    ylim(-200000, 0.0)
    legend(bbox_to_anchor=(1.05, 1), loc=2)
savefig("QFE_Ne8.pdf")

In [ ]:
print len(QFe)

In [ ]:
#QFe = [10.7-0.1*i for i in range(28)]# SF = 1
#Ne =[1.4,152.4,755.6,1816.3,3268.0,5019.3,7115.7,9577.2,12420,15451,18599,21902,25308,28962,33051,37432,42049,46855,51813,56886,61971,67101,72275,77582,82953,88442,94162,100075]# SF = 1

QFe = array([9.3-0.1*i for i in range(14)])
Ne = array([0.4,96.5,751.3,2071.6,4033.5,6606.7,9787,13491,17751,22418,27528,33055,38973,45216])
slope, intercept, r_value, p_value, std_err = stats.linregress(QFe[0:3], Ne[0:3])
xplot = linspace(4.0,9.0,100)
yplot = intercept + slope * xplot
slope2, intercept2, r_value, p_value, std_err = stats.linregress(QFe[4:9], Ne[4:9])
xplot2 = linspace(4.0,10.0,100)
yplot2 = intercept2 + slope2 * xplot2
slope3, intercept3, r_value, p_value, std_err = stats.linregress(QFe[10:14], Ne[10:14])
xplot3 = linspace(4.0,10.0,100)
yplot3 = intercept3 + slope3 * xplot3
#fit = polyfit(Ne, QFe,2)
#print fit
#yplot4 = linspace(0.0,50000.0,500)
#xplot4 = fit[2] + fit[1] * yplot4 + fit[0] * yplot4 * yplot4
fit = polyfit(QFe, Ne,2)
print fit
xplot4 = linspace(7.0,10.0,500)
yplot4 = fit[2] + fit[1] * xplot4 + fit[0] * xplot4 * xplot4
A = fit[0]; B = fit[1]; C= fit[2]
for i,ne in enumerate(Ne):
    qfe_model = (-B - sqrt(B**2 - 4.0 * A * (C-ne))) / (2.0*A)
    print "Ne = %.1f, QFe = %.4f, QFe_model = %.4f"%(ne,QFe[i],qfe_model)

figure()
title("Number of Electrons vs Quasi-Fermi Level")
scatter(QFe,Ne)
#plot(xplot, yplot, color = 'red', ls = '--')
#plot(xplot2, yplot2, color = 'green', ls = '--')
#plot(xplot3, yplot3, color = 'blue', ls = '--')
plot(xplot4, yplot4, color = 'blue', ls = '--')

xlabel("Quasi-Fermi Level(V)")
ylabel("Number of Electrons")
#text(6.0,275000,"Slope = %f"%slope, color='red')
#text(6.0,250000,"Intercept = %f"%(intercept/slope), color='red')
#text(6.0,200000,"Slope = %f"%slope2,color='green')
#text(6.0,175000,"Intercept = %f"%(intercept2/slope2),color='green')
#text(7.5,125000,"Slope = %f"%slope3,color='blue')
#text(7.5,100000,"Intercept = %f"%(intercept3/slope3),color='blue')
#ylim(50000,100000)
xlim(7.5,9.5)

savefig("QFE_Ne5.pdf")

In [ ]:
from scipy.special import lambertw
ktq = .015
clf()
figure()
xplot = linspace(-20.0, 20.0,500)
for mult in [0.0,1.0,10.0]:
    yplot = lambertw(exp(xplot)-mult)
    slope, intercept, r_value, p_value, std_err = stats.linregress(xplot[0:10], log(yplot[0:10]))
    print slope, intercept, r_value
    slope, intercept, r_value, p_value, std_err = stats.linregress(xplot[490:500], yplot[490:500])
    print slope, intercept, r_value
    plot(xplot, log10(yplot))
savefig("Temp.pdf")